In [ ]:
import numpy as np
import pandas as pd
import re
import requests
import holoviews as hv
import hvplot.pandas
import panel as pn
import pint
from cytoolz import dissoc

In [ ]:
%load_ext autoreload
%autoreload 2
import util

In [ ]:
hv.extension("bokeh")

# Spectral response

In [ ]:
def read_spectrum(id_, bins=None):
    df = pd.read_csv(
        f"https://www.fpbase.org/spectra_csv/?q={id_}", index_col="wavelength"
    ).squeeze()
    if bins is not None:
        df = util.interpolate_dataframe(df, bins)
    return df


def get_fpbase_spectra(bins=None):
    fpbase_entries = requests.get("https://www.fpbase.org/api/proteins/spectra/").json()
    fps = {}
    for fpbase_entry in fpbase_entries:
        fp = {}
        spectra = []
        for spectrum_type in fpbase_entry["spectra"]:
            state_name = re.sub(r"^default_", "", spectrum_type["state"])
            for k, v in spectrum_type.items():
                if k == "state":
                    continue
                elif k == "data":
                    spectra.append(
                        pd.DataFrame(v, columns=["wavelength", state_name]).set_index(
                            "wavelength"
                        )
                    )
                else:
                    fp[f"{state_name}_{k}"] = v
        df = pd.concat(spectra, axis=1)
        if bins is not None:
            df = util.interpolate_dataframe(df, bins)
        fp["spectra"] = df
        fps[fpbase_entry["name"]] = fp
    return fps

In [ ]:
bins = np.arange(300, 1000)

In [ ]:
%%time
sources = {"SOLIS-3C": "7016", "SOLIS-565C": "7004"}

dichroics = {
    "Chroma ZT514rdc": "560",
    "Chroma ZT532rdc": "645",
    "Chroma T550lpxr": "803",
    "Chroma T556lpxr": "658",
    "Chroma ZT561rdc": "523",
    "Chroma ZT561rdc-xr": "708",
    "Chroma ZT568rdc": "604",
    "Chroma T570lpxr": "427",
    "Chroma T590lpxr": "593",
    "Chroma ZT594rdc": "569",
    "Chroma T600lpxr": "609",
    "Chroma T610lpxr": "601",
    "Chroma ZT633rdc": "668",
    "Chroma T635lpxr": "616",
    "Chroma ZT640rdc": "439",
}

longpass_filters = {
    "Chroma ET520LP": "760",
    "Chroma ET525lp": "530",
    "Chroma ET542lp": "461",
    "Chroma ET570lp": "487",
    "Chroma ET575lp": "620",
    "Chroma ET590lp": "805",
    "Chroma ET610lp": "350",
    "Chroma RET638lp": "2855",
    "Chroma ET655lp": "683",
    "Chroma ET665lp": "573",
}

sources = {name: read_spectrum(q, bins=bins) for name, q in sources.items()}
dichroics = {name: read_spectrum(q, bins=bins) for name, q in dichroics.items()}
longpass_filters = {
    name: read_spectrum(q, bins=bins) for name, q in longpass_filters.items()
}

In [ ]:
%%time
fps = get_fpbase_spectra(bins=bins)

In [ ]:
def _filter_plot(fp_name, dc_name, lp_name):
    return (
        fps[fp_name]["spectra"].fillna(0).hvplot()
        * dichroics[dc_name].hvplot()
        * longpass_filters[lp_name].hvplot()
    )


def filter_plot(
    fp_names=fps.keys(), dc_names=dichroics.keys(), lp_names=longpass_filters.keys()
):
    layout = pn.interact(
        _filter_plot, fp_name=fp_names, dc_name=dc_names, lp_name=lp_names
    )
    return pn.Row(pn.Column("## Filters", layout[0]), layout[1])

## Filters

In [ ]:
def combine_filter(dc, lp, threshold=0.9, max_wavelength=700):
    dc = dc.loc[:max_wavelength]
    lp = lp.loc[:max_wavelength]
    log_dc = np.log10(dc)
    log_lp = np.log10(lp)
    # highest wavelength that gives >90% excitation (dichroic reflectance)
    ex_cutoff = (dc > 1 - threshold)[::-1].idxmin()
    # highest wavelength that gives <90% transmission (dichroic+lp)
    em_cutoff = (log_dc + log_lp > np.log10(threshold)).idxmax()
    # gap between those wavelengths
    gap = em_cutoff - ex_cutoff
    # highest OD below excitation cutoff
    worst_rejection = (log_dc + log_lp).loc[:ex_cutoff].max()
    return pd.Series(
        {
            "ex_cutoff": ex_cutoff,
            "em_cutoff": em_cutoff,
            "gap": gap,
            "worst_rejection": worst_rejection,
        }
    )


filter_combinations = pd.concat(
    {
        dc_name: pd.concat(
            {
                lp_name: combine_filter(dc_spectrum, lp_spectrum)
                for lp_name, lp_spectrum in longpass_filters.items()
            },
            axis=1,
        ).T
        for dc_name, dc_spectrum in dichroics.items()
    }
).sort_values("ex_cutoff")

In [ ]:
acceptable_filter_combinations = filter_combinations[
    (filter_combinations["gap"] < 30) & (filter_combinations["worst_rejection"] < -5.5)
]

In [ ]:
# for each dichroic, pick the longpass that minimizes the gap
selected_filter_combinations = (
    acceptable_filter_combinations.groupby(level=0)
    .apply(lambda x: x.loc[[x["gap"].idxmin()]])
    .droplevel(0)
    .sort_values("ex_cutoff")
    .index.values
)

## FPs

In [ ]:
acceptable_filter_combinations

In [ ]:
selected_filter_combinations

In [ ]:
def show_heatmap(df):
    with pd.option_context(
        "display.max_rows", None, "display.max_columns", None
    ):  # more options can be specified also
        display(
            df.style.format(precision=2).background_gradient(cmap="RdPu", axis=None)
        )


def fp_efficiency(dc, lp, ex, em, ex_cutoff, em_cutoff):
    ex_efficiency = ((1 - dc) * (ex / ex.sum())).sum()
    em_efficiency = (dc * lp * (em / em.sum())).sum()
    ex_threshold = 0
    em_threshold = 0
    # margin from ex/em peak to cutoff
    return pd.Series(
        {
            "ex_efficiency": ex_efficiency,
            # "ex_threshold": ex_threshold,
            "em_efficiency": em_efficiency,
            # "em_threshold": em_threshold,
        }
    )


def evaluate_filter_combinations(
    filter_combo_names, fp_names, dichroics, longpass_filters, filter_combinations, fps
):
    d = {}
    for filter_combo in filter_combo_names:
        dc = dichroics[filter_combo[0]]
        lp = longpass_filters[filter_combo[1]]
        dd = {}
        for fp_name in fp_names:
            ex = fps[fp_name]["spectra"]["ex"]
            em = fps[fp_name]["spectra"]["em"]
            dd[fp_name] = (
                fp_efficiency(
                    dc,
                    lp,
                    ex,
                    em,
                    filter_combinations[filter_combo]["ex_cutoff"],
                    filter_combinations[filter_combo]["em_cutoff"],
                )
                .to_frame()
                .T
            )
        d[filter_combo] = pd.concat(dd, axis=1).swaplevel(axis=1).sort_index(axis=1)
    return pd.concat(d, axis=0).droplevel(-1)

In [ ]:
selected_fps = [
    "mScarlet-I",
    "mScarlet-H",
    "mCherry",
    "mCherry2",
    "mKate2",
    "E2-Crimson",
]
filter_metrics = evaluate_filter_combinations(
    selected_filter_combinations,
    selected_fps,
    dichroics,
    longpass_filters,
    filter_combinations,
    fps,
)

In [ ]:
show_heatmap(filter_metrics)

In [ ]:
filter_plot(fp_names=selected_fps)